In [ ]:
!git clone https://github.com/facebookresearch/MUSE.git

Cloning into 'MUSE'...
remote: Enumerating objects: 239, done.
remote: Total 239 (delta 0), reused 0 (delta 0), pack-reused 239
Receiving objects: 100% (239/239), 215.77 KiB | 19.62 MiB/s, done.
Resolving deltas: 100% (136/136), done.


/content/MUSE/data


In [ ]:
# %cd MUSE/data/
!chmod +x ./get_evaluation.sh
!./get_evaluation.sh

In [ ]:
!pip install fasttext
!apt-get install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

In [ ]:
!curl -Lo wiki.es.vec https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2474M  100 2474M    0     0  24.5M      0  0:01:40  0:01:40 --:--:-- 23.4M


In [ ]:
!curl -Lo wiki.en.vec https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6291M  100 6291M    0     0  24.9M      0  0:04:12  0:04:12 --:--:-- 25.0M


In [ ]:
%cd ../

/content/MUSE


In [ ]:
%cd data/

/content/MUSE/data


In [ ]:
!wget https://dl.fbaipublicfiles.com/arrival/wordsim.tar.gz

--2021-05-01 09:55:30--  https://dl.fbaipublicfiles.com/arrival/wordsim.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204540 (200K) [application/gzip]
Saving to: ‘wordsim.tar.gz’

wordsim.tar.gz      100%[===================>] 199.75K   807KB/s    in 0.2s    

2021-05-01 09:55:30 (807 KB/s) - ‘wordsim.tar.gz’ saved [204540/204540]



In [ ]:
import shutil

# shutil.rmtree('/content/MUSE/__MACOSX')
# shutil.rmtree('/content/MUSE/dumped')
shutil.rmtree('/content/MUSE/data/monolingual')

In [ ]:
!tar -xf wordsim.tar.gz
!mv wordsim monolingual 

In [ ]:
%cd ../

/content/MUSE


In [ ]:
!python supervised.py --src_lang en --tgt_lang es --src_emb data/wiki.en.vec --tgt_emb data/wiki.es.vec --n_refinement 10 --dico_train default

INFO - 05/01/21 10:13:31 - 0:00:00 - ============ Initialized logger ============
INFO - 05/01/21 10:13:31 - 0:00:00 - cuda: True
                                     dico_build: S2T&T2S
                                     dico_eval: default
                                     dico_max_rank: 10000
                                     dico_max_size: 0
                                     dico_method: csls_knn_10
                                     dico_min_size: 0
                                     dico_threshold: 0
                                     dico_train: default
                                     emb_dim: 300
                                     exp_id: 
                                     exp_name: debug
                                     exp_path: /content/MUSE/dumped/debug/rtik0dkt21
                                     export: txt
                                     max_vocab: 200000
                                     n_refinement: 10
                         

In [ ]:
!curl -Lo ./data/wiki.fr.vec https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fr.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2886M  100 2886M    0     0  22.7M      0  0:02:06  0:02:06 --:--:-- 24.2M


In [ ]:
!python supervised.py --src_lang es --tgt_lang fr --src_emb data/wiki.es.vec --tgt_emb data/wiki.fr.vec --n_refinement 10 --dico_train default

INFO - 05/01/21 10:43:19 - 0:00:00 - ============ Initialized logger ============
INFO - 05/01/21 10:43:19 - 0:00:00 - cuda: True
                                     dico_build: S2T&T2S
                                     dico_eval: default
                                     dico_max_rank: 10000
                                     dico_max_size: 0
                                     dico_method: csls_knn_10
                                     dico_min_size: 0
                                     dico_threshold: 0
                                     dico_train: default
                                     emb_dim: 300
                                     exp_id: 
                                     exp_name: debug
                                     exp_path: /content/MUSE/dumped/debug/2etvwrpu0m
                                     export: txt
                                     max_vocab: 200000
                                     n_refinement: 10
                         

In [ ]:
# testing 
import io
import numpy as np

In [ ]:
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [ ]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [ ]:
import torch
matrx_en_es = torch.from_numpy(torch.load('/content/MUSE/dumped/debug/rtik0dkt21/best_mapping.pth'))
matrx_en_es = matrx_en_es.detach().numpy()

matrx_es_fr = torch.from_numpy(torch.load('/content/MUSE/dumped/debug/2etvwrpu0m/best_mapping.pth'))
matrx_es_fr = matrx_es_fr.detach().numpy()

In [ ]:
en_path = '/content/MUSE/data/wiki.en.vec'
nmax = 200000  # maximum number of word embeddings to load
en_embeddings, en_id2word, en_word2id = load_vec(en_path, nmax)

In [ ]:
es_path = '/content/MUSE/data/wiki.es.vec'
es_embeddings, es_id2word, es_word2id = load_vec(es_path, nmax)

In [ ]:
fr_path = '/content/MUSE/data/wiki.fr.vec'
fr_embeddings, fr_id2word, fr_word2id = load_vec(fr_path, nmax)

In [ ]:
# src_embeddings_2 = matrx_es_fr.dot(src_embeddings_2.transpose()).transpose()
# en mappiing to es
en_embeddings = matrx_en_es.dot(en_embeddings.transpose()).transpose()
# en mapping to fr
# src_embeddings = matrx_es_fr.dot(src_embeddings.transpose()).transpose()

In [ ]:
# printing nearest neighbors in the source space
#test with source en
src_word = 'cat'
get_nn(src_word, en_embeddings, en_id2word, en_embeddings, en_id2word, K=5)

Nearest neighbors of "cat":
1.0000 - cat
0.7322 - cats
0.6453 - kitten
0.6381 - dog
0.6218 - kittens


In [ ]:
# printing nearest neighbors in the source space
#test with source es
src_word = 'gato'
get_nn(src_word, es_embeddings, es_id2word, es_embeddings, es_id2word, K=5)

Nearest neighbors of "gato":
1.0000 - gato
0.6868 - gatos
0.6618 - conejo
0.6526 - perro
0.6358 - gatito


In [ ]:
src_word = 'cat'
get_nn(src_word, en_embeddings, en_id2word, es_embeddings, es_id2word, K=5)

Nearest neighbors of "cat":
0.6239 - gato
0.5517 - perro
0.5393 - gatito
0.4895 - conejo
0.4846 - gorila


In [ ]:
#test mapping en -> fr
en_embeddings = matrx_es_fr.dot(en_embeddings.transpose()).transpose()

In [ ]:
src_word = 'cat'
get_nn(src_word, en_embeddings, en_id2word, fr_embeddings, fr_id2word, K=5)

Nearest neighbors of "cat":
0.6133 - chat
0.5643 - cat
0.5226 - chien
0.5104 - lapin
0.5008 - caniche


In [ ]:
while True:
  pass

KeyboardInterrupt: ignored